<a href="https://colab.research.google.com/github/Adhithyan-1004/Chatbot/blob/main/Movie_Genre_Classification.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install pandas scikit-learn matplotlib seaborn nltk joblib


In [ ]:
import pandas as pd
import numpy as np
import re
import string
import nltk
import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.naive_bayes import MultinomialNB
from sklearn.svm import LinearSVC
from sklearn.metrics import classification_report, confusion_matrix

nltk.download('stopwords')
from nltk.corpus import stopwords

# ----------- Load and Parse Data ----------- #
def parse_line(line):
    parts = line.strip().split(" ::: ")
    if len(parts) == 4:
        return {'id': parts[0], 'title': parts[1], 'genre': parts[2], 'plot': parts[3]}
    return None

with open("train_data.txt", "r", encoding="utf-8") as f:
    parsed_data = [parse_line(line) for line in f if parse_line(line)]

df = pd.DataFrame(parsed_data)

# ----------- Preprocess Text ----------- #
def clean_text(text):
    text = text.lower()
    text = re.sub(r'\d+', '', text)
    text = text.translate(str.maketrans('', '', string.punctuation))
    tokens = text.split()
    tokens = [word for word in tokens if word not in stopwords.words('english')]
    return " ".join(tokens)

df['clean_plot'] = df['plot'].apply(clean_text)

# ----------- TF-IDF Vectorization ----------- #
vectorizer = TfidfVectorizer(max_features=5000)
X = vectorizer.fit_transform(df['clean_plot'])
y = df['genre']

# ----------- Train-Test Split ----------- #
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# ----------- Train Models ----------- #
models = {
    "Logistic Regression": LogisticRegression(max_iter=1000),
    "Naive Bayes": MultinomialNB(),
    "SVM": LinearSVC()
}

for name, model in models.items():
    print(f"\nTraining model: {name}")
    model.fit(X_train, y_train)
    y_pred = model.predict(X_test)

    print("Classification Report:")
    print(classification_report(y_test, y_pred))

    cm = confusion_matrix(y_test, y_pred)
    plt.figure(figsize=(6, 4))
    sns.heatmap(cm, annot=True, fmt='d', cmap='Blues')
    plt.title(f'Confusion Matrix - {name}')
    plt.xlabel('Predicted')
    plt.ylabel('Actual')
    plt.show()

# ----------- Save Model ----------- #
import joblib
joblib.dump(models["Logistic Regression"], "logistic_model.pkl")
joblib.dump(vectorizer, "tfidf_vectorizer.pkl")


[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.



Training model: Logistic Regression
